In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.utils import resample
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import TSNE
%matplotlib inline

In [2]:
df_pca = pd.read_csv("pca_data.csv",index_col='Unnamed: 0')

In [3]:
df_pca.head()

,Principal Component 1,Principal Component 2,Principal Component 3,Principal Component 4,Principal Component 5,Principal Component 6,Principal Component 7,Principal Component 8,Principal Component 9,Principal Component 10,...,Principal Component 82,Principal Component 83,Principal Component 84,Principal Component 85,Principal Component 86,Principal Component 87,Principal Component 88,Principal Component 89,Principal Component 90,Release Decade
0,-0.150333,-0.071206,0.035277,0.078159,0.010680,0.013304,-0.019755,-0.026418,-0.018521,-0.046058,...,0.001183,-0.001290,-0.004443,-0.000603,0.001508,-0.002278,-0.002729,-0.000815,-0.002403,2000
1,-0.127587,-0.042643,-0.039528,0.030215,0.055388,-0.029260,-0.044218,-0.043042,0.024067,-0.017534,...,0.005864,0.008839,0.003453,0.005332,0.006288,-0.000495,0.002620,0.000646,-0.002889,2000
2,-0.173302,-0.032196,0.000596,0.036082,0.019383,-0.006957,-0.007135,-0.040764,-0.023660,-0.005569,...,-0.001461,-0.003321,-0.001130,-0.002359,0.003970,0.003404,-0.001038,-0.003000,0.004577,2000
3,-0.104943,-0.114886,0.054070,-0.024135,0.055518,-0.013094,-0.010121,-0.011146,0.000877,-0.002956,...,0.005481,0.002250,0.009814,0.006775,0.000168,0.000743,0.007096,-0.001337,-0.002772,2000
4,-0.167919,-0.048979,0.009335,0.051351,0.022709,0.004066,-0.054789,-0.016897,0.027231,-0.020771,...,0.005635,-0.004517,-0.001881,-0.001931,0.001328,0.006725,-0.005325,0.003686,0.000467,2000


In [4]:
X,y = df_pca.drop("Release Decade",axis=1),df_pca["Release Decade"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0,stratify = y)

In [6]:
# define early stopping callback
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')

Using TensorFlow backend.


In [7]:
from keras.callbacks import ModelCheckpoint
modelcheck = ModelCheckpoint('C:\\Users\\Lenovo\\Desktop\\ML\\Data Mining and Decision Models\\Decision Models Assignment\\checkpoints\\classifier_conv_checkpoint.hdf5',monitor='val_acc',verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [8]:
from time import time
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir="logs/{}".format(time()),histogram_freq=1, batch_size=32, write_graph=True, write_grads=True,write_images=True)

In [9]:
callbacks_list=[tensorboard,earlystop,modelcheck]

In [10]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense,Dropout,Activation

In [11]:
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)

In [12]:
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

In [13]:
print(X_train.shape, X_test.shape)

(412276, 90, 1) (103069, 90, 1)


In [14]:
# Initialising the CNN
classifier_conv= Sequential()

# Step 1 - Convolution
classifier_conv.add(Conv1D(32,10, padding='same',input_shape =X_train.shape[1:3],activation='relu'))
# Step 2 - Pooling
classifier_conv.add(MaxPooling1D(pool_size=2))
# Adding a second convolutional layer
classifier_conv.add(Conv1D(32, 5 , activation = 'relu'))
classifier_conv.add(MaxPooling1D(pool_size = 2))

# Step 3 - Flattening
classifier_conv.add(Flatten())
classifier_conv.add(Dropout(0.4))

# Step 4 - Full connection
classifier_conv.add(Dense(units = 128, activation = 'relu'))
classifier_conv.add(Dense(units = len(df_pca['Release Decade'].unique())))
classifier_conv.add(Activation('softmax'))

# Compiling the CNN
classifier_conv.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 90, 32)            352       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 45, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 41, 32)            5152      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 20, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 640)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               82048     
__________

In [15]:
classifier_conv.fit(X_train, pd.get_dummies(y_train), batch_size = 10, epochs = 5,callbacks=callbacks_list,validation_data=(X_test,pd.get_dummies(y_test)))

Train on 412276 samples, validate on 103069 samples
Epoch 1/5
412276/412276 [==============================] - 573s 1ms/step - loss: 1.0430 - acc: 0.6121 - val_loss: 0.9838 - val_acc: 0.6290
Epoch 2/5
412276/412276 [==============================] - 534s 1ms/step - loss: 1.0083 - acc: 0.6227 - val_loss: 0.9698 - val_acc: 0.6336
Epoch 3/5
412276/412276 [==============================] - 502s 1ms/step - loss: 1.0001 - acc: 0.6256 - val_loss: 0.9695 - val_acc: 0.6335
Epoch 4/5
412276/412276 [==============================] - 413s 1ms/step - loss: 0.9953 - acc: 0.6273 - val_loss: 0.9681 - val_acc: 0.6361
Epoch 5/5
412276/412276 [==============================] - 499s 1ms/step - loss: 0.9938 - acc: 0.6282 - val_loss: 0.9686 - val_acc: 0.6386
